## Import

In [21]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [23]:
# Load historical treasury data
file_path = "./data/Anleihen und Indizes historische Zinsen.xlsx"
df_anleihen = pd.read_excel(file_path, sheet_name="Anleihen", skiprows=2)

# Load historical indices data
df_etf = pd.read_excel(file_path, sheet_name="Indizes")

df_anleihen = df_anleihen.drop(df_anleihen.columns[[6, 11, 12, 13, 14, 15]], axis=1)
df_anleihen.columns = ["date", "australia", "canada", "france", "germany", "japan", "spain", "switzerland", "uk", "us"]

df_anleihen.head(10)

,date,australia,canada,france,germany,japan,spain,switzerland,uk,us
0,1954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.401667
1,1955,NaN,3.189167,NaN,NaN,NaN,NaN,2.982500,NaN,2.816667
2,1956,NaN,3.605833,NaN,NaN,NaN,NaN,3.125000,NaN,3.182500
3,1957,NaN,4.125000,NaN,7.516667,NaN,NaN,3.667500,NaN,3.647500
4,1958,NaN,4.115833,NaN,6.783333,NaN,NaN,3.153333,NaN,3.315833
5,1959,NaN,5.048333,NaN,5.766667,NaN,NaN,3.076667,NaN,4.333333
6,1960,NaN,5.188334,5.663333,6.425000,NaN,NaN,3.092500,5.855834,4.116667
7,1961,NaN,5.058333,5.519166,5.875000,NaN,NaN,2.955833,6.264167,3.882500
8,1962,NaN,5.099167,5.434166,5.908333,NaN,NaN,3.124167,5.764167,3.945833
9,1963,NaN,5.090833,5.338333,6.050000,NaN,NaN,3.246667,5.165833,4.002500


In [24]:
# Define the range of years
x_range = range(1, 31)  

# Initialize the structure for the JSON
json_structure = {
    "assets": []
}

# Iterate through all columns except 'date'
for column_name in df_anleihen.columns:
    if column_name == 'date':
        continue  

    growth_column = df_anleihen[column_name]
    date_column = df_anleihen['date']

    # Add a new asset entry for the current column
    asset_entry = {
        "category": "Anleihe",
        "label": f"{column_name}",
        "kennung": f"Anleihe {column_name}",
        "id": f"Anleihe {column_name}",
        "data": []  # This will hold all the data arrays for this column
    }

    # Iterate through all values of x
    for x in x_range:
        highest_value = float('-inf')
        lowest_value = float('inf')
        highest_year = None
        lowest_year = None
        highest_values_list = []  
        lowest_values_list = []  
        median_values_list = []  
        values_with_years = [] 

        # Iterate through all rows where there are numbers
        valid_growths = growth_column.dropna()
        valid_dates = date_column[~growth_column.isna()] 

        for i in range(len(valid_growths) - x + 1): 
            value = 1
            growth_array = []  
            for growth in valid_growths[i:i + x]:  
                value *= (1 + growth / 100)  
                growth_array.append(growth)  
            
            values_with_years.append((value, valid_dates.iloc[i], growth_array))  
            if value > highest_value:
                highest_value = value
                highest_year = valid_dates.iloc[i]
                highest_values_list = growth_array  
            if value < lowest_value:
                lowest_value = value
                lowest_year = valid_dates.iloc[i]
                lowest_values_list = growth_array 

        # Calculate the median value and find its corresponding year and array
        if values_with_years:  
            sorted_values = sorted(values_with_years, key=lambda v: v[0])
            median_index = len(sorted_values) // 2
            median_value, median_year, median_values_list = sorted_values[median_index]

            # Create dictionaries for max, median, and min arrays
            max_dict = {
                "type": f"max",
                "duration": f"{x}",
                "year": str(highest_year),
                "growth_array": highest_values_list
            }
            median_dict = {
                "type": f"median",
                "duration": f"{x}",
                "year": str(median_year),
                "growth_array": median_values_list
            }
            min_dict = {
                "type": f"min",
                "duration": f"{x}",
                "year": str(lowest_year),
                "growth_array": lowest_values_list
            }

            # Append the dictionaries to the "data" key in the asset entry
            asset_entry["data"].extend([max_dict, median_dict, min_dict])
        else:
            print(f"No valid values for column '{column_name}' and x = {x}. Skipping...")

    # Append the asset entry to the JSON structure
    json_structure["assets"].append(asset_entry)

# Save the JSON structure to a file
with open("export/data.json", "w") as json_file:
    json.dump(json_structure, json_file, separators=(",", ":"))  # Minify JSON by removing spaces

print("JSON file saved successfully!")

JSON file saved successfully!
